In [ ]:
%pip install -U ragatouille -q
%pip install -U langchain -q
%pip uninstall --y faiss-cpu & pip install faiss-gpu --force-reinstall -q   # przy małej liczbie plików/konwersacji wystarczy faiss-cpu. Przy większej - faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 717.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.9

In [5]:
import datetime
import json
from typing import List
from pathlib import Path
from pprint import pprint

from langchain_core.documents import Document

In [6]:
# plik z histrią konwersacji ściągamy z ustawień ChatGPT. z zapisanego foldery wyłapujemy plik conversatios.json.

file_path='/content/drive/MyDrive/Colab Notebooks/chatgpt_data/conversations.json' # plik z historią konwersacji

chat_data = json.loads(Path(file_path).read_text())
titles = [data['title'] for data in chat_data]

In [7]:
# pierwsze cztery tytuły konwersacji (od najnowszej)

titles[:4]

#titles[-4]   # ostatnie cztery konwersacje

['Regulamin pracy zoptymalizowany.',
 'Zegarek nowoczesny minimalista',
 'Oracle GoldenGate - replikacja danych',
 'ScrapeGraphAI Tool Summary']

In [8]:
# przygotowanie wszystkich konwersacji dla Langchain (klasa Document)

all_docs  = []

documents = []

for data in chat_data:

    title = data['title']
    messages = data['mapping']
    documents.append([messages[key] for idx, key in enumerate(messages) if not (idx==0 and messages[key]['message']['author']['role']=='system')])

for i in range(0, len(chat_data)):

    data = chat_data[i]
    messages = documents[i]
    for d in messages:
        if d["message"] != None:

            try:
                date = datetime.datetime.fromtimestamp(d["message"]["create_time"]).strftime("%Y-%m-%d %H:%M:%S")
                sender = d["message"]["author"]["role"] if d["message"]["author"] else "unknown"
                text = d["message"]["content"]["parts"][0]
                metadata = {
                    "title": data['title'],
                    "author": sender,
                    "date": date
                }

                all_docs.append(Document(page_content=text, metadata=metadata))

            except:
                ValueError('Error')

In [10]:
# przygotowanie historii konwersacji w formie listy dla retrivera na bazie modelu Colbert2

history = []

for title in titles:
    string = "Title: " + title + "\n"
    for doc in all_docs:
        if doc.metadata['title'] == title:
            conversation = '\n' + doc.metadata['author'] + '\n' + doc.metadata['date'] + '\n' + doc.page_content
            string += conversation
    history.append(string)

len(history)

481

In [11]:
# załadowanie modelu Colbert2

from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
# przygotowanie indeksu dla historii konwersacji

index_path = RAG.index(
    index_name="my_index",
    collection=history,
    max_document_length=512,
    split_documents=True,
    use_faiss=True,
)



[May 17, 10:50:13] #> Creating directory .ragatouille/colbert/indexes/my_index 


[May 17, 10:50:14] [0] 		 #> Encoding 3623 passages..
[May 17, 10:51:02] [0] 		 avg_doclen_est = 348.0165710449219 	 len(local_sample) = 3,623
[May 17, 10:51:02] [0] 		 Creating 16,384 partitions.
[May 17, 10:51:02] [0] 		 *Estimated* 1,260,864 embeddings.
[May 17, 10:51:02] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/my_index/plan.json ..
[May 17, 10:52:11] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[May 17, 10:53:44] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[0.026, 0.025, 0.026, 0.023, 0.023, 0.025, 0.023, 0.023, 0.024, 0.024, 0.023, 0.025, 0.024, 0.023, 0.024, 0.024, 0.023, 0.024, 0.023, 0.024, 0.024, 0.024, 0.023, 0.024, 0.022, 0.024, 0.024, 0.024, 0.025, 0.025, 0.024, 0.027, 0.024, 0.023, 0.023, 0.023, 0.024, 0.024, 0.023, 0.028, 0.026, 0.025, 0.024, 0.025, 

0it [00:00, ?it/s]

[May 17, 10:55:19] [0] 		 #> Encoding 3623 passages..


1it [00:52, 52.11s/it]
100%|██████████| 1/1 [00:00<00:00, 198.11it/s]

[May 17, 10:56:11] #> Optimizing IVF to store map from centroids to list of pids..
[May 17, 10:56:11] #> Building the emb2pid mapping..
[May 17, 10:56:11] len(emb2pid) = 1260864



100%|██████████| 16384/16384 [00:00<00:00, 36871.32it/s]

[May 17, 10:56:12] #> Saved optimized IVF to .ragatouille/colbert/indexes/my_index/ivf.pid.pt


Done indexing!


In [19]:
# przykładowe odpytanie indeksu

results = RAG.search(query="Czy Oracle GoldenGate służy do replikacji danych?", k=3)
results

[{'content': '2. **Replikacja w czasie rzeczywistym**: Oracle GoldenGate umożliwia replikację danych transakcyjnych na żywo, co zapewnia aktualność danych w systemach rozproszonych.\n   \n3. **Transformacja i filtrowanie danych**: Oferuje możliwość transformacji danych podczas ich przesyłu, co pozwala na dostosowanie przepływu danych do specyficznych potrzeb biznesowych.\n   \n4. **Wysoka wydajność i skalowalność**: GoldenGate jest zaprojektowane do pracy z minimalnym wpływem na wydajność systemów źródłowych i docelowych, co jest kluczowe dla przedsiębiorstw o dużej skali operacji.\n   \n5. **Zabezpieczenia i szyfrowanie**: Zapewnia bezpieczeństwo przesyłanych danych poprzez zaawansowane mechanizmy szyfrowania i autoryzacji.\n\nOracle GoldenGate jest cenione za swoją elastyczność i wydajność w zarządzaniu i synchronizacji danych w różnorodnych środowiskach IT, co czyni je popularnym rozwiązaniem w dziedzinie zarządzania danymi w czasie rzeczywistym.\nuser\n2024-05-11 10:54:51\nUser\nZa

In [ ]:
# przygotowanie bazy wektorej FAISS oraz ensambling modeli dla retrivera - cdn ... :-)
